In [1]:
import tensorflow.keras as keras
import tensorflow.compat.v2 as tf
from utils.read_mnist import read_mnist
import lenet

In [2]:
train_x, train_y, test_x, test_y = read_mnist(one_hot=True, standard=True)

In [3]:
model = lenet.model(regular=0.00001)

In [4]:
tensorboardlog = r"trainlogs\\"
trainlogdir = tensorboardlog
scalarlogdir = tensorboardlog
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=trainlogdir)
file_writer = tf.summary.create_file_writer(scalarlogdir)
file_writer.set_as_default()

In [5]:
def lr_schedule(epoch):
    """Returns a custom learning rate that decreases as epochs progress.    """
    learning_rate = 0.02
    if epoch > 2:
        learning_rate = 0.002
    if epoch > 4:
        learning_rate = 0.0001
    if epoch > 6:
        learning_rate = 0.00001
    if epoch > 8:
        learning_rate = 0.00001

    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [6]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
earlystops = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [7]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [13]:
model.state_updates

[]

In [9]:
b = tf.get_tensor_by_name('conv2d/kernel:0')
print(b)

AttributeError: module 'tensorflow._api.v2.compat.v2' has no attribute 'get_tensor_by_name'

In [8]:
callbacks = [tensorboard_callback,
             earlystops, lr_callback]

In [9]:
history = model.fit(x=train_x, y=train_y,
                    batch_size=64,
                    epochs=10,
                    validation_data=(test_x, test_y),
                    shuffle=True,
                    callbacks=callbacks)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 14s 233us/sample - loss: 1.1772 - accuracy: 0.5951 - val_loss: 0.2529 - val_accuracy: 0.9306
Epoch 2/10
60000/60000 [==============================] - 12s 203us/sample - loss: 0.4210 - accuracy: 0.8747 - val_loss: 0.1551 - val_accuracy: 0.9573
Epoch 3/10
60000/60000 [==============================] - 12s 196us/sample - loss: 0.2934 - accuracy: 0.9179 - val_loss: 0.1144 - val_accuracy: 0.9673
Epoch 4/10
60000/60000 [==============================] - 12s 195us/sample - loss: 0.2335 - accuracy: 0.9358 - val_loss: 0.1049 - val_accuracy: 0.9707
Epoch 5/10
60000/60000 [==============================] - 12s 196us/sample - loss: 0.2308 - accuracy: 0.9367 - val_loss: 0.1028 - val_accuracy: 0.9715
Epoch 6/10
60000/60000 [==============================] - 12s 200us/sample - loss: 0.2263 - accuracy: 0.9392 - val_loss: 0.1018 - val_accuracy: 0.9721
Epoch 7/10
60000/60000 [====================

In [10]:
model.evaluate(test_x, test_y)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.10133332449793815, 0.9725]

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840 

In [12]:
model.save('mymodel.h5')

In [13]:
model.save('mymodel')

W0926 18:28:42.604125 18556 deprecation.py:506] From G:\anaconda\envs\tensorflow2\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
